# 01 - BaT Scraping

Scrape completed Porsche 911 auctions from Bring a Trailer.

**Outputs:**
- `data/raw/bat_listings.parquet` - Raw scraped data

In [1]:
import logging
import pandas as pd
from pathlib import Path

from price_analysis.constants import SCRAPING_DEFAULTS
from price_analysis.scraping import fetch_auctions, validate_scraped_data, DataQualityError
from price_analysis.scraping.bat import listings_to_dataframe

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [2]:
# Paths
DATA_DIR = Path("../data")
RAW_DIR = DATA_DIR / "raw"
RAW_DIR.mkdir(parents=True, exist_ok=True)

OUTPUT_PATH = RAW_DIR / "bat_listings.parquet"

## Configure Scraping

Adjust search queries and pagination as needed.

In [3]:
# Search queries - multiple generation-specific searches to get more results
# BaT seems to limit how many results a single search returns
QUERIES = [
    "Porsche 996",  # 1999-2004 (996.1 and 996.2)
    "Porsche 997",  # 2005-2012 (997.1 and 997.2)
    "Porsche 991",  # 2012-2019 (991.1 and 991.2)
    "Porsche 992",  # 2020+ (992.1 and 992.2)
    "Porsche 911",  # Catch air-cooled + anything else missed
]

# Scraping parameters (from centralized constants)
MAX_CLICKS = SCRAPING_DEFAULTS["max_clicks"]
DELAY = SCRAPING_DEFAULTS["delay"]
HEADLESS = SCRAPING_DEFAULTS["headless"]

## Debug: Inspect BaT Page Structure (Optional)

Skip this section - fixture files already saved to `tests/fixtures/`.
Only re-run if BaT changes their DOM structure.

In [4]:
# Debug: Save sample pages for selector inspection
import time
from price_analysis.scraping import create_driver, save_debug_page

# Create driver (set HEADLESS=False above to watch)
driver = create_driver(headless=HEADLESS)

try:
    # 1. Fetch search results page
    search_url = "https://bringatrailer.com/auctions/results/?s=Porsche+911"
    logger.info(f"Fetching search page: {search_url}")
    driver.get(search_url)
    time.sleep(3)  # Wait for JS to load
    save_debug_page(driver, "bat_search_page", output_dir="../tests/fixtures")

    # 2. Fetch a specific listing page (992.1)
    listing_url = "https://bringatrailer.com/listing/2020-porsche-911-carrera-4s-coupe-26/"
    logger.info(f"Fetching listing: {listing_url}")
    driver.get(listing_url)
    time.sleep(3)
    save_debug_page(driver, "bat_listing_992", output_dir="../tests/fixtures")

    # 3. Fetch another listing (997.2) for variety
    listing_url_997 = "https://bringatrailer.com/listing/2009-porsche-911-carrera-4s-coupe-28/"
    logger.info(f"Fetching listing: {listing_url_997}")
    driver.get(listing_url_997)
    time.sleep(3)
    save_debug_page(driver, "bat_listing_997", output_dir="../tests/fixtures")

    logger.info("Debug pages saved to tests/fixtures/")

finally:
    driver.quit()

INFO:WDM:====== WebDriver manager ======
INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Driver [/Users/flatljan/.wdm/drivers/chromedriver/mac64/143.0.7499.169/chromedriver-mac-arm64/chromedriver] found in cache
INFO:__main__:Fetching search page: https://bringatrailer.com/auctions/results/?s=Porsche+911
INFO:price_analysis.scraping.bat:Saved debug page to ../tests/fixtures/bat_search_page.html
INFO:__main__:Fetching listing: https://bringatrailer.com/listing/2020-porsche-911-carrera-4s-coupe-26/
INFO:price_analysis.scraping.bat:Saved debug page to ../tests/fixtures/bat_listing_992.html
INFO:__main__:Fetching listing: https://bringatrailer.com/listing/2009-porsche-911-carrera-4s-coupe-28/
INFO:price_analysis.scraping.bat:Saved debug page to ../tests/fixtures/bat_listing_997.html
INFO:__main__:Debug pages saved to tests/fixtures/


## Run Scraper

This will take a while depending on MAX_PAGES. Each page + listing takes ~3-5 seconds.

In [5]:
# Load existing URLs for incremental scraping (skip already-fetched listings)
existing_urls: set[str] = set()
if OUTPUT_PATH.exists():
    existing_df = pd.read_parquet(OUTPUT_PATH)
    existing_urls = set(existing_df["listing_url"])
    logger.info(f"Loaded {len(existing_urls)} existing URLs - will skip these")

all_listings = []

for query in QUERIES:
    logger.info(f"Scraping: {query}")
    listings = fetch_auctions(
        query=query,
        max_clicks=MAX_CLICKS,
        delay=DELAY,
        headless=HEADLESS,
        existing_urls=existing_urls,
    )
    all_listings.extend(listings)
    logger.info(f"Found {len(listings)} NEW listings for '{query}'")

logger.info(f"Total NEW listings scraped: {len(all_listings)}")

INFO:__main__:Loaded 1615 existing URLs - will skip these
INFO:__main__:Scraping: Porsche 996
INFO:WDM:====== WebDriver manager ======
INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Driver [/Users/flatljan/.wdm/drivers/chromedriver/mac64/143.0.7499.169/chromedriver-mac-arm64/chromedriver] found in cache
INFO:price_analysis.scraping.bat:Fetching search results: https://bringatrailer.com/auctions/results/?s=Porsche+996
INFO:price_analysis.scraping.bat:Clicked 'Show More' (1/50)
INFO:price_analysis.scraping.bat:Clicked 'Show More' (2/50)
INFO:price_analysis.scraping.bat:Clicked 'Show More' (3/50)
INFO:price_analysis.scraping.bat:Clicked 'Show More' (4/50)
INFO:price_analysis.scraping.bat:Clicked 'Show More' (5/50)
INFO:price_analysis.scraping.bat:Clicked 'Show More' (6/50)
INFO:price_analysis.scraping.bat:Clicked 'Show More' (7/50)
INFO:price_analysis.scraping.bat:Clicked 'Show More' (8/50)
INFO:price_analysis

KeyboardInterrupt: 

## Convert to DataFrame and Save

In [ ]:
df = listings_to_dataframe(all_listings)

# Run data quality checks and filter low-price listings (<$10k)
# Returns filtered DataFrame, raises DataQualityError if checks fail
df = validate_scraped_data(df)

display(df.head(10))
print(f"\nShape: {df.shape}")

INFO:price_analysis.scraping.bat:Filtered 164 listings below $10,000 (likely parts/salvage)
INFO:price_analysis.scraping.bat:Data quality checks passed: 62 listings, 62 with prices


,listing_url,title_raw,sale_price,sale_date,model_year,generation,trim,transmission,mileage,color,location
28,https://bringatrailer.com/listing/2002-porsche...,2002 Porsche 911 Carrera Targa 6-Speed,32500,2025-12-26,2002.0,996.2,Targa,Manual,110000.0,None,"Tampa, Florida 33606"
34,https://bringatrailer.com/listing/2001-porsche...,24k-Mile 2001 Porsche 911 Carrera Cabriolet 6-...,30250,2025-12-26,2001.0,996.1,Carrera,Manual,24000.0,Arctic Silver Metallic,"Darien, Wisconsin 53114"
50,https://bringatrailer.com/listing/2006-porsche...,2006 Porsche 911 Carrera Coupe 6-Speed,37000,2025-12-26,2006.0,997.1,Carrera,Manual,79000.0,Arctic Silver Metallic,"Orlando, Florida 32806"
68,https://bringatrailer.com/listing/2011-porsche...,2011 Porsche 911 Turbo S Coupe,91000,2025-12-26,2011.0,997.2,Turbo S,PDK,55000.0,Black,"Manchester, New Hampshire 03102"
71,https://bringatrailer.com/listing/2017-porsche...,20k-Mile 2017 Porsche 911 Turbo S Coupe,161000,2024-11-26,2017.0,991.2,Turbo S,PDK,20000.0,GT Silver Metallic,"Newtown, Pennsylvania 18940"
75,https://bringatrailer.com/listing/2015-porsche...,2015 Porsche 911 GT3 Cup,105911,2024-12-13,2015.0,991.1,GT3,None,19000.0,None,"Corona, California 92883"
77,https://bringatrailer.com/listing/2018-porsche...,"2,300-Mile 2018 Porsche 911 GT2 RS Weissach",405000,2024-12-10,2018.0,991.2,GT2 RS,PDK,2300.0,Miami Blue,"Scottsdale, Arizona 85255"
78,https://bringatrailer.com/listing/2012-porsche...,38k-Mile 2012 Porsche 911 Carrera Cabriolet,53000,2024-11-26,2012.0,997.2,Carrera,PDK,38000.0,Carrara White,"Weehawken, New Jersey 07086"
83,https://bringatrailer.com/listing/2017-porsche...,12k-Mile 2017 Porsche 911 Targa 4S 7-Speed,134000,2024-12-01,2017.0,991.2,Targa 4S,Manual,12000.0,Black,"Sumter, South Carolina 29154"
86,https://bringatrailer.com/listing/2016-porsche...,354-Mile 2016 Porsche 911 GT3 RS,215000,2024-11-26,2016.0,991.2,GT3 RS,None,354.0,Ultraviolet,"Turtle Creek, Pennsylvania 15145"



Shape: (62, 11)


In [ ]:
# Append to existing if present, otherwise create new
if OUTPUT_PATH.exists():
    existing = pd.read_parquet(OUTPUT_PATH)
    df = pd.concat([existing, df], ignore_index=True)
    df = df.drop_duplicates(subset=["listing_url"], keep="last")
    logger.info(f"Merged with existing data: {len(df)} total listings")

df.to_parquet(OUTPUT_PATH, index=False)
logger.info(f"Saved to {OUTPUT_PATH}")

INFO:__main__:Merged with existing data: 1615 total listings
INFO:__main__:Saved to ../data/raw/bat_listings.parquet


## Quick Inspection

In [ ]:
print("Counts by generation:")
display(df["generation"].value_counts(dropna=False))

Counts by generation:


generation
997.1    274
992.1    241
991.2    206
996.1    187
996.2    170
None     166
997.2    158
991.1    121
992.2     92
Name: count, dtype: int64

In [ ]:
print("Counts by trim:")
display(df["trim"].value_counts(dropna=False))

Counts by trim:


trim
Carrera        381
Carrera S      194
Turbo          186
Carrera 4S     166
None           137
Turbo S        109
GT3            101
Carrera 4       79
GT3 RS          76
GT3 Touring     65
Targa 4S        39
Targa           36
Targa 4         30
GT2 RS          16
Name: count, dtype: int64

In [ ]:
print("Counts by transmission:")
display(df["transmission"].value_counts(dropna=False))

Counts by transmission:


transmission
Manual       953
PDK          503
None          63
Tiptronic     51
Automatic     45
Name: count, dtype: int64

In [ ]:
# Check parsing quality - how many have all required fields?
required = ["sale_price", "model_year", "generation", "trim", "transmission", "mileage"]
complete = df[required].notna().all(axis=1).sum()
print(
    f"\nListings with all required fields: {complete} / {len(df)} ({complete / len(df) * 100:.1f}%)"
)


Listings with all required fields: 1360 / 1615 (84.2%)


In [ ]:
# Sample some listings to verify parsing
print("Sample listings for manual verification:")
sample = df.sample(min(10, len(df)), random_state=42)
for _, row in sample.iterrows():
    print(f"\n{row['title_raw']}")
    print(
        f"  Parsed: {row['model_year']} {row['generation']} {row['trim']} ({row['transmission']})"
    )
    print(f"  Price: ${row['sale_price']:,}" if pd.notna(row["sale_price"]) else "  Price: N/A")
    print(f"  Mileage: {row['mileage']:,}" if pd.notna(row["mileage"]) else "  Mileage: N/A")

Sample listings for manual verification:

2003 Porsche 911 Turbo Coupe X50 6-Speed
  Parsed: 2003.0 996.2 Turbo (Manual)
  Price: $99,518
  Mileage: 53,000.0

2024 Porsche 911 Carrera GTS Coupe 7-Speed
  Parsed: 2024.0 992.2 Carrera (Manual)
  Price: $151,000
  Mileage: 21.0

38k-Mile 2013 Porsche 911 Carrera S Coupe
  Parsed: 2013.0 991.1 Carrera S (PDK)
  Price: $87,500
  Mileage: 38,000.0

2021 Porsche 911 Carrera S Coupe
  Parsed: 2021.0 992.1 Carrera S (PDK)
  Price: $113,000
  Mileage: 19,000.0

Euro Porsche 997 GT3 Lightweight Bucket Seats
  Parsed: nan None GT3 (Manual)
  Price: $15,250
  Mileage: N/A

1973.5 Porsche 911T Coupe 5-Speed
  Parsed: 1973.0 None None (Manual)
  Price: $78,000
  Mileage: 9,000.0

2023 Porsche 911 GT3 RS Weissach
  Parsed: 2023.0 992.1 GT3 RS (PDK)
  Price: $415,000
  Mileage: 1,900.0

41k-Mile 2001 Porsche 911 Carrera 4 Cabriolet 6-Speed
  Parsed: 2001.0 996.1 Carrera 4 (Manual)
  Price: $29,911
  Mileage: 41,000.0

9k-Mile 2015 Porsche 911 GT3
  Par